In [ ]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, LSTM, InputLayer, Bidirectional,Dropout

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
train_text = train['excerpt']
test_text = test['excerpt']

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_text, target_vocab_size=2**13)
print(tokenizer.subwords[:50])

In [ ]:
print(train_text[0][:100])
print(tokenizer.encode(train_text[0][:100]))

In [ ]:
train_x = []
for text in train_text:
    train_x.append(tokenizer.encode(text))
test_x = []
for text in test_text:
    test_x.append(tokenizer.encode(text))

max_len = 0
for i in train_x:
    if max_len < len(i):
        max_len = len(i)
for i in test_x:
    if max_len < len(i):
        max_len = len(i)
print(max_len)

X_train=pad_sequences(train_x, maxlen=max_len, padding='post')
X_test = pad_sequences(test_x, maxlen=max_len, padding='post')

X_train = np.array(X_train).reshape(2834,351,1)
X_test = np.array(X_test).reshape(7,351,1)
y = np.array(train['target']).reshape(-1,1)
X_train.shape, X_test.shape

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(351, return_sequences=False), input_shape=X_train.shape[1:]))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(X_train,y,validation_split=0.3, epochs=30, batch_size=64,verbose=1)

In [ ]:
predict = model.predict(X_test)
submit = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submit['target'] = predict
submit.to_csv('submission.csv',index=None)